# Mathematical Model Order Reduction

This notebook shows how to create a reduced model using mathematical model order reduction (MOR). We will reduce a computationally demanding Fokker-Planck-Mckean-Vlasov type meanfield model of FitzHugh-Nagumo neurons. The model is a nonlinear partial differential equation, therefore it must be studied numerically.

With mathematical MOR, we can create a reduced model that _approximates_ the original, computationally demanding model so that _no assumptions_ or _simplifications_ are done. In other words, the reduced model is able to model _every_ variable of the original system. This is in contrast to using simplified models such as the integrate and fire model, to approximate a complex system like the Hodgkin-Huxley model.

## Model implementation

This model has been published in Baladron, J., Fasoli, D., Faugeras, O., & Touboul, J. (2012). Mean-field description and propagation of chaos in networks of Hodgkin-Huxley and FitzHugh-Nagumo neurons. The Journal of Mathematical Neuroscience, 2(1), 10. The model is defined as 
$$
\partial_t p(t,V,W,Y)
	= \frac{1}{2} \sigma_J^2 \bar{y}^2(t) \frac{\partial^2}{\partial V^2} \bigg [(V-V_{rev})^2 p(t,V,W,Y)\bigg ]
	+ \frac{1}{2} \frac{\partial^2}{\partial Y^2} \bigg[ \sigma_Y^2(V,Y) p(t,V,W,Y) \bigg]
	+ \frac{1}{2} \sigma_{ext}^2 \frac{\partial^2}{\partial V^2} \bigg[ p(t,V,W,Y) \bigg]
	- \frac{\partial}{\partial V} \bigg[ \bigg( V-\frac{V^3}{3}
	- W + I_{ext}(t) + \bar{J}(V-V_{rev})\bar{y}(t) \bigg) p(t,V,W,Y) \bigg]
	- \frac{\partial}{\partial W} \bigg[a(V+b-cW)p(t,V,W,Y)\bigg] 
	- \frac{\partial}{\partial Y} \bigg[\bigg(\alpha_r S(V)(1-Y) - \alpha_d Y\bigg) p(t,V,W,Y)\bigg] 
$$


In the FitzHugh-Nagumo neurons, there are 2 dependent variables, V (membrane voltage) and W (voltage recovery),
additionally our network needs a synaptic conductance variable Y. Our system is then 3 dimensional and the domain is a cube. To visualize the state of the system at any given time, we will calculate the marginal probability distribution over Y. 

Before our model can be simulated, the partial differential equation needs to be discretized.
The model is implemented with 4th order central difference discretization, but we need to set
the number of discretization points for the dependent variables of the model. 


In [ ]:
# Begin by importing everything we will need
import time

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from model_reduction.models.fitzhugh_nagumo_meanfield import FitzHughNagumoMeanfield
from model_reduction.deim import DEIM

In [ ]:
# Discretize the PD, too many points will make you quickly run out of memory
# but a fine grid with many points is needed for the simulation to be accurate!
points_v = 20
points_w = 20
points_y = 20

print('The original model will have {} equations (dimensions) to solve at every timestep'.format(
    points_v*points_w*points_y))

# Initialize the model
model = FitzHughNagumoMeanfield(points_v, points_w, points_y)

# Initial values come from the Gaussian distribution, we can fetch them from our model
initial_values = model.get_initial_values()

# Let's plot the initial state of the system to confirm the model was created successfully
# We need to create a grid for the 3D visualization
%matplotlib inline
density, _ = model.compute_VW_density(initial_values)
w_grid, v_grid = np.meshgrid(model.discrete_w, model.discrete_v)
initial_fig = plt.figure()
initial_ax = initial_fig.gca(projection='3d')
initial_ax.plot_surface(v_grid, w_grid, density)
plt.show()

In [ ]:
# We are ready to simulate the original, unreduced model, let's go for t_end seconds of biological time.
# We need to define simulation time and a small timestep for numerical simulation.
# Additionally, we will measure how long the simulation takes. It will take a while, a plot will be drawn 
# once the simulation ends.
t_start = 0.0
t_end = 2.0
timestep = 0.01

start_time = time.time()
solutions = model.simulate(initial_values, t_start, t_end, timestep)
original_model_time = time.time() - start_time
print('Time elapsed: {}s'.format(original_model_time))

# Let's plot the state of the system at the end of the simulation
%matplotlib inline
original_density, _ = model.compute_VW_density(solutions[-1])
original_fig = plt.figure()
original_ax = original_fig.gca(projection='3d')
original_ax.plot_surface(v_grid, w_grid, original_density)
original_ax.set_title('Original model')
plt.show()

In [ ]:
# Now let's create a reduced model and see how it compares to the original model in results and computation time
# We can choose the dimension of the model, compare that number to original model.
dimensions = 5
reduced_model = DEIM(model)
reduced_model.reduce(dimensions, dimensions)

# We will simulate the reduced model with exactly the same parameters as the original model
start_time = time.time()
reduced_solutions = reduced_model.simulate(initial_values, t_start, t_end, timestep)
reduced_model_time = time.time() - start_time
print('Time elapsed: {}s'.format(reduced_model_time))

# Simulation of the reduced model took place in a small dimensional subspace! 
# This is what makes the simulation fast. To visualize the result in our original space,
# we need to transform the solutions. This way we will obtain an approximation of _every_
# variable in the original model (now it is points of the or cube domain, but it could be anything,
# for example ion channel currents).
reduced_solutions = np.dot(reduced_model.pod_basis, np.array(reduced_solutions).T)

# See how the reduced model looks, and visualize the difference between the two models
%matplotlib inline
reduced_density, _ = model.compute_VW_density(reduced_solutions[:, -1])
reduced_fig = plt.figure()
reduced_ax = reduced_fig.gca(projection='3d')
reduced_ax.plot_surface(v_grid, w_grid, density)
reduced_ax.set_title('Reduced model')

difference = np.abs(np.subtract(original_density, reduced_density))
error_fig = plt.figure()
error_ax = error_fig.gca(projection='3d')
error_ax.plot_surface(v_grid, w_grid, difference)
error_ax.set_title('Absolute difference')
#error_ax.set_zlim(0.0, original_density.max())
plt.show()

print('The reduced model was over {t} times faster to simulate with maximum absolute difference of {e}'.format(
    t=int(original_model_time/reduced_model_time), e=np.max(difference)))